Modelos ML, en este archivo se realizan las operaciones necesarias para aplicar el modelo de Machine Learning y recomendaicones

In [ ]:
# Se importan las librerías necesarias para trabajar
import pandas as pd # Librería de pandas para el uso de dataframes
# librerías de SKlearn para aplicar los modelos de ML
from sklearn.metrics.pairwise        import cosine_similarity
from sklearn.metrics.pairwise        import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
# Se procede a cargar los archivos de csv limpios en un dataframe
df_Steam_Games =pd.read_csv('..\\..\\Archivos Limpios\\output_steam_games_limpio.csv') # Se carga en el dataframe nuestro archivo de items


In [25]:
# Se hacen algunos trabajos adicionales en el dataframe de steam games
df_Steam_Games = df_Steam_Games.loc[:, ["specs", "item_id", "name"]] # Seleccionamos las columnas necesarias
df_Steam_Games["item_id"] = df_Steam_Games["item_id"].astype(int) # Se convierte el item id a entero
df_Steam_Games['specs'] = df_Steam_Games['specs'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace("'", '')) # Se eliminan los caracteres especiales de la columna specs

Modelo de machine learning para recomendación de juegos

Creación de un Vectorizador de Texto (CountVectorizer):

Propósito: Convertir texto (en este caso, la columna 'specs') en vectores numéricos.

Proceso: El vectorizador asigna un número a cada palabra única presente en el texto y cuenta su frecuencia. Cada documento (en este caso, cada juego) se representa como un vector donde cada posición corresponde a la frecuencia de una palabra.

Importancia: Esta representación vectorial es necesaria para calcular la similitud del coseno, ya que este método trabaja con vectores numéricos.

In [26]:
# Se obtiene solo una muestra para el procesamiento de la información
df_RecomendacionxJuego = df_Steam_Games.head(4000)
df_RecomendacionxUsuario = df_Steam_Games.head(4000)

In [27]:
# Crear un vectorizador de texto
cv = CountVectorizer() # Se instancia el contador de vectores
vectores_juego = cv.fit_transform(df_RecomendacionxJuego['specs']).toarray()
vectores_Usuario = cv.fit_transform(df_RecomendacionxUsuario['specs']).toarray()


Cálculo de la Similitud del Coseno entre Vectores:

Propósito: Evaluar la similitud entre los vectores numéricos de diferentes juegos.

Proceso: Se utiliza la métrica de similitud del coseno, que mide el coseno del ángulo entre dos vectores. Cuanto más cercano a 1, más similares son los vectores.

Importancia: La similitud del coseno es clave para determinar qué tan parecidos son los juegos entre sí. Esto se utiliza para generar recomendaciones, ya que los juegos con vectores similares son considerados como recomendaciones potenciales.

In [28]:
# Calcular la similitud del coseno entre vectores
similitud_juego = cosine_similarity(vectores_juego)
similitud_usuario = cosine_similarity(vectores_Usuario)

A continuación, se genera una función (recomendacion) que toma como entrada el ID de un juego y devuelve una lista de los cinco juegos más recomendados basándose en la similitud del coseno entre vectores.La función devuelve la lista de títulos recomendados.

In [29]:
# Se genera la función de recomendación
def recomendacionPorJuego(juego): # Se define la función de recomendación
    #Se busca el índice del juego en el DataFrame original. Este índice es utilizado para acceder a la fila correspondiente en la matriz de similitud.
    indice_juego = df_Steam_Games[df_Steam_Games["item_id"] == juego].index[0]
    
    #Cálculo de Similitudes: Se obtienen las distancias de similitud entre el juego de entrada y todos los demás juegos en el conjunto de datos. 
    distancia = similitud_juego[indice_juego]
    
    #Las distancias se ordenan de manera descendente, y se seleccionan los cinco juegos más similares (excluyendo el juego de entrada)
    lista_juegos = sorted(list(enumerate(distancia)), reverse=True, key=lambda x: x[1])[1:6]
    
    #Salida. Devuelve la lista de títulos recomendados.
    recomendaciones = [df_Steam_Games.iloc[i[0]]['name'] for i in lista_juegos]
    
    return recomendaciones # Se retorna la lista de titulos

In [30]:
# Aplicar la función a la columna 'item_id' y crear una nueva columna 'Recomendaciones'
df_RecomendacionxJuego['Recomendaciones'] = df_RecomendacionxJuego['item_id'].apply(recomendacion)

C:\Users\Windows10\AppData\Local\Temp\ipykernel_7820\868046866.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RecomendacionxJuego['Recomendaciones'] = df_RecomendacionxJuego['item_id'].apply(recomendacion)


In [31]:
# Eliminar columnas innecesarias para disminuir el tamaño del archivo de salida
df_RecomendacionxJuego.drop(columns=['name', 'specs'], inplace=True)

C:\Users\Windows10\AppData\Local\Temp\ipykernel_7820\2697776106.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RecomendacionxJuego.drop(columns=['name', 'specs'], inplace=True)


In [34]:
# Se guarda el DataFrame resultante en un nuevo archivo CSV
df_RecomendacionxJuego.to_csv('..\\..\\Modelos_Consultas_Funciones\\Recomendación_por_Juego.csv', index=False)